<a href="https://colab.research.google.com/github/matheusmf1/aws-bid-machine/blob/main/rhs_dataAnalisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install awswrangler

In [244]:
import pandas as pd
import awswrangler as wr

In [269]:
# fileName = "/content/drive/MyDrive/Colab Notebooks/155917_8_1095_20201207223020.json"
fileName = "/content/drive/MyDrive/Colab Notebooks/155917_3_1091_20201207220101.json"

data = pd.read_json( fileName )

data

,licitacoes
licitacao,[{'palavrasEncontradasAgrupadasXml': {'palavra...


In [ ]:
def getJsonDataWr():
  bucket = 's3://rhs-xml-test/'
  # filePath = event['Records'][0]['s3']['object']['key']
  # filePath = 'JSON/155917_3_1091_20201207220101.json'
  filePath = "JSON/155917_8_1095_20201207223020.json"

  path = f'{bucket}{filePath}'

  json_data = wr.s3.read_json( [ path ] )
  return json_data

def getJsonData( event ):
    # filePath = event['Records'][0]['s3']['object']['key']
    # filePath = "JSON/155917_8_1095_20201207223020.json"
    filePath = "JSON/155917_3_1091_20201207220101.json"
    
    s3 = boto3.resource( 's3' )
    
    content_object = s3.Object( 'rhs-xml-test', filePath )

    file_content = content_object.get()['Body'].read()
 
    json_data = json.loads( file_content )

    return json_data


In [270]:
licitacaoType = type(data['licitacoes']['licitacao'])

if licitacaoType == dict:
  
  mainData = data['licitacoes']['licitacao']
  df_new = prepareDataSet( mainData ) 

elif licitacaoType == list:

  df_new = pd.DataFrame()
  
  for licitacao in data['licitacoes']['licitacao']:
    df_new = df_new.append( prepareDataSet( licitacao ), ignore_index = True )

In [271]:
df_new

,objeto,modalidade,registro,orgao,prazoTipo,prazoData,prazoHora,endereco,cidade,uf,telefone,site,palavraEncontrada,links,trechos
0,UASG: 200109 PREGAO ELETRONICO - CONTRATACAO...,PE/21/2020,10235516/11590,Departamento de Policia Rodoviaria Federal,Prazo para Abertura,17/12/2020,09:00,"Spo, Quadra 3, Lote 5 Complexo Sede da PRF, As...",Brasilia,DF,Nao informado pela fonte,None,"Armazenamento,AVA,Backup,CRM,EAD,Google,Hardwa...",https://www.licitacao.com.br/editais/IA-MARE/2...,"originÃ¡rios da contrataÃ§Ã£o, recolhendo-os a..."
1,UASG: 135058 PREGAO ELETRONICO - CONTRATACAO...,PE/14/2020,10246947/11590,EMBRAPA - Empresa Brasileira de Pesquisa Agrop...,Prazo para Abertura das Propostas,18/12/2020,09:00,http://www.comprasnet.gov.br,Brasilia,DF,Nao informado pela fonte,None,"Armazenamento,Cloud,ERP,Nuvem,SAAS",https://www.licitacao.com.br/editais/IA-MARE/2...,"materiais reciclÃ¡veis, de forma a garantir a ..."
2,UASG: 135036 PREGAO ELETRONICO - FORMACAO DE...,PE/38/2020,10246938/11590,EMBRAPA - Empresa Brasileira de Pesquisa Agrop...,Prazo para Abertura das Propostas,17/12/2020,09:00,"Rod. GO 462, Km 12",SANTO ANTONIO DE GOIAS,GO,(62) 3533-2190,None,"Armazenamento,Backup,Big Data,Hardware,Microso...",https://www.licitacao.com.br/editais/IA-MARE/2...,Module) para <em>armazenamento</em> de chaves ...


In [265]:
def prepareDataSet( data ):

  df = pd.DataFrame( data )

  # Create new Columns
  createNewColumns( df )

  # Drop nested columns
  df.drop( ['palavrasEncontradasAgrupadasXml', 'linksAgrupadosXml', 'trechosEncontradosAgrupadosXml' ], axis = 1, inplace = True  )

  df.reset_index( drop = True, inplace = True )
  df.drop_duplicates( inplace = True )

  return df

In [ ]:
def createNewColumns( df ):
  
  if type( df['palavrasEncontradasAgrupadasXml']['palavra'] ) == str:
    
    df['palavraEncontrada'] = df['palavrasEncontradasAgrupadasXml']['palavra']

  elif type(df['palavrasEncontradasAgrupadasXml']['palavra']) == list:

    df['palavraEncontrada'] = ",".join( df['palavrasEncontradasAgrupadasXml']['palavra'] )  


  if type( df['linksAgrupadosXml']['link'] ) == str:
    
    df['links'] = df['linksAgrupadosXml']['link']

  elif ( type( df['linksAgrupadosXml']['link'] ) == list ):
    
    df['links'] = ",".join( df['linksAgrupadosXml']['link'] )  


  if type( df['trechosEncontradosAgrupadosXml']['trecho'] ) == str:
    
    df['trechos'] = df['trechosEncontradosAgrupadosXml']['trecho']

  elif ( type( df['trechosEncontradosAgrupadosXml']['trecho'] ) == list ):
   
    df['trechos'] = ",".join( df['trechosEncontradosAgrupadosXml']['trecho'] )